In [ ]:
#!pip install pynrrd opencv-python

In [ ]:
import nrrd
import glob
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
labels = [1,2,3]

In [ ]:
def split_into_consecutive_sublists(lst):
    result = []
    current_sublist = [lst[0]]

    for i in range(1, len(lst)):
        if lst[i] == lst[i-1] + 1:
            current_sublist.append(lst[i])
        else:
            result.append(current_sublist)
            current_sublist = [lst[i]]
    
    result.append(current_sublist)
    return result

In [ ]:
study = "Baseline"

for filePath in glob.glob("D:\Research2021\Rad_TM\data\pat data-20231016T160300Z-001\pat data\done/*/" + study + "/"): # change the path to the folder containing the .nrrd files for annotation
    files = [f for f in os.listdir(filePath) if os.path.isfile(os.path.join(filePath, f)) and f.endswith('.nrrd')]
    imagePath = os.path.join(filePath, files[0])
    maskPath = os.path.join(filePath, 'Segmentation.nrrd')
    datamask, headermask = nrrd.read(maskPath)
    dataimage, headerimage = nrrd.read(imagePath)
    print(filePath)

    
    for l in labels:
        print("labels: ", l)
        slices_0 = []
        slices_1 = []
        slices_2 = []
        # extract roi from image where mask is 1 and save as 2D image
        image = np.where(datamask == l, dataimage, 0)
        # print((np.where(datamask == l)[0]))
        # print((np.where(datamask == l)[1]))
        # print(np.where(datamask == l)[2])

        if l == 1:
            path = study + "/cisternrois/"
        elif l == 2:
            path = study + "/bonerois/"
        elif l == 3:
            path = study + "/ccrois/"

        slices_0 = np.unique(np.where(datamask == l)[0]).tolist()

        if len(slices_0) > 1:
            for sl in slices_0:
                #print(sl)
                slices_1 = np.unique(np.where(datamask[[sl],:,:] == l)[1]).tolist()
                #print(slices_1)
                slices_2 = np.unique(np.where(datamask[[sl],:,:] == l)[2]).tolist()
                #print(slices_2)
        
                # slice image and mask based on the slices
                image_ = image[:,slices_1,:]
                mask_ = datamask[:,slices_1,:]

                #print(image_.shape)

                image_ = image_[:,:,slices_2]
                mask_ = mask_[:,:,slices_2]
                #print(image_.shape)

                image_ = image_[[sl],:,:]
                mask_ = mask_[[sl],:,:]

                #print(image_.shape)

                # save all images as .npy files         

                file_name = filePath.split(os.sep)[-3] + str(sl) + "_img.npy"
                mask_file_name = filePath.split(os.sep)[-3] + str(sl) + "_mask.npy"
                print(file_name)

                image_ = image_[0,:,:]
                mask_ = mask_[0,:,:]
                #print(image_)
                image_ = cv2.flip(image_, 1)
                mask_ = cv2.flip(mask_, 1)
                image_ = cv2.rotate(image_, cv2.ROTATE_90_CLOCKWISE)

                # # Remove all rows in image_ that are all 0
                image_ = image_[~np.all(image_ == 0, axis=1)]
                print(image_.shape)

                #print(image_.shape)
                # reshape image and mask to 13x13

                # image_ = cv2.resize(image_, (13,13), interpolation = cv2.INTER_AREA)
                # mask_ = cv2.resize(mask_, (13,13), interpolation = cv2.INTER_AREA)
        
                image_name = filePath.split(os.sep)[-3] + "_" + str(sl) + "_img.png"
                mask_name = filePath.split(os.sep)[-3] + "_" + str(sl) + "_mask.png"
                print(image_name)
                cv2.imwrite(path + image_name, image_)
                #cv2.imwrite(path + mask_name, mask_)

                # reshape image and mask to 1x13x13
                image_ = np.expand_dims(image_, axis=0)
                mask_ = np.expand_dims(mask_, axis=0)

                print(image_)

                np.save(path + file_name, image_)
                np.save(path + mask_file_name, mask_)

        else:
            slices_1 = np.unique(np.where(datamask[slices_0,:,:] == l)[1]).tolist()
            #print(slices_1)
            mul_sl = split_into_consecutive_sublists(slices_1)
            #print("mul..........................................................")
            #print(mul_sl)
            index = 0
            for sl in mul_sl:
                slices_2 = np.unique(np.where(datamask[slices_0,:,:] == l)[2]).tolist()
                #print(slices_2)
        
                # slice image and mask based on the slices

                image_ = image[:,:,slices_2]
                mask_ = datamask[:,:,slices_2]

                image_ = image_[:,sl,:]
                mask_ = mask_[:,sl,:]

                image_ = image_[slices_0,:,:]
                mask_ = mask_[slices_0,:,:]

                

                # save all images as .npy files         

                file_name = filePath.split(os.sep)[-3] + "_" + str(index) + "_img.npy"
                mask_file_name = filePath.split(os.sep)[-3] + "_" + str(index) + "_mask.npy"

                image_ = image_[0,:,:]
                mask_ = mask_[0,:,:]
                
                image_ = cv2.flip(image_, 1)
                mask_ = cv2.flip(mask_, 1)
                image_ = cv2.rotate(image_, cv2.ROTATE_90_CLOCKWISE)

                # image_ = cv2.resize(image_, (13,13), interpolation = cv2.INTER_AREA)
                # mask_ = cv2.resize(mask_, (13,13), interpolation = cv2.INTER_AREA)
               
                # # Remove all rows in image_ that are all 0
                image_ = image_[~np.all(image_ == 0, axis=1)]
                print(image_.shape)
        
                image_name = filePath.split(os.sep)[-3] + "_" + str(index) + "_img.png"
                mask_name = filePath.split(os.sep)[-3] + "_" + str(index) + "_mask.png"
                cv2.imwrite(path + image_name, image_)
                ##cv2.imwrite(path + mask_name, mask_)
                
                image_ = np.expand_dims(image_, axis=0)
                mask_ = np.expand_dims(mask_, axis=0)

                print(image_)

                np.save(path + file_name, image_)
                np.save(path + mask_file_name, mask_)

                index += 1

    

In [ ]:
study = "Controls"

for filePath in glob.glob("D:/Research2021/Rad_TM/data/pat data-20231016T160300Z-001/controls/T1/*/"):
    files = [f for f in os.listdir(filePath) if os.path.isfile(os.path.join(filePath, f)) and f.endswith('.nrrd')]
    imagePath = os.path.join(filePath, files[0])
    maskPath = os.path.join(filePath, 'Segmentation.seg.nrrd')
    datamask, headermask = nrrd.read(maskPath)
    dataimage, headerimage = nrrd.read(imagePath)
    print(filePath, maskPath, imagePath)

    
    for l in labels:
        print("labels: ", l)
        slices_0 = []
        slices_1 = []
        slices_2 = []
        # extract roi from image where mask is 1 and save as 2D image
        image = np.where(datamask == l, dataimage, 0)
        # print((np.where(datamask == l)[0]))
        # print((np.where(datamask == l)[1]))
        # print(np.where(datamask == l)[2])

        if l == 1:
            path = study + "/cisternrois/"
        elif l == 2:
            path = study + "/bonerois/"
        elif l == 3:
            path = study + "/ccrois/"

        slices_0 = np.unique(np.where(datamask == l)[0]).tolist()

        if len(slices_0) > 1:
            for sl in slices_0:
                #print(sl)
                slices_1 = np.unique(np.where(datamask[[sl],:,:] == l)[1]).tolist()
                #print(slices_1)
                slices_2 = np.unique(np.where(datamask[[sl],:,:] == l)[2]).tolist()
                #print(slices_2)
        
                # slice image and mask based on the slices
                image_ = image[:,slices_1,:]
                mask_ = datamask[:,slices_1,:]

                #print(image_.shape)

                image_ = image_[:,:,slices_2]
                mask_ = mask_[:,:,slices_2]
                #print(image_.shape)

                image_ = image_[[sl],:,:]
                mask_ = mask_[[sl],:,:]

                #print(image_.shape)

                # save all images as .npy files         

                file_name = filePath.split(os.sep)[-2] + str(sl) + "_img.npy"
                mask_file_name = filePath.split(os.sep)[-2] + str(sl) + "_mask.npy"
                print(file_name)

                image_ = image_[0,:,:]
                mask_ = mask_[0,:,:]
                #print(image_)
                image_ = cv2.flip(image_, 1)
                mask_ = cv2.flip(mask_, 1)
                image_ = cv2.rotate(image_, cv2.ROTATE_90_CLOCKWISE)

                # # Remove all rows in image_ that are all 0
                image_ = image_[~np.all(image_ == 0, axis=1)]
                print(image_.shape)

                #print(image_.shape)
                # reshape image and mask to 13x13

                # image_ = cv2.resize(image_, (13,13), interpolation = cv2.INTER_AREA)
                # mask_ = cv2.resize(mask_, (13,13), interpolation = cv2.INTER_AREA)
        
                image_name = filePath.split(os.sep)[-2] + "_" + str(sl) + "_img.png"
                mask_name = filePath.split(os.sep)[-2] + "_" + str(sl) + "_mask.png"
                print(image_name)
                cv2.imwrite(path + image_name, image_)
                #cv2.imwrite(path + mask_name, mask_)

                # reshape image and mask to 1x13x13
                image_ = np.expand_dims(image_, axis=0)
                mask_ = np.expand_dims(mask_, axis=0)

                print(image_)

                np.save(path + file_name, image_)
                np.save(path + mask_file_name, mask_)

        else:
            slices_1 = np.unique(np.where(datamask[slices_0,:,:] == l)[1]).tolist()
            #print(slices_1)
            mul_sl = split_into_consecutive_sublists(slices_1)
            #print("mul..........................................................")
            #print(mul_sl)
            index = 0
            for sl in mul_sl:
                slices_2 = np.unique(np.where(datamask[slices_0,:,:] == l)[2]).tolist()
                #print(slices_2)
        
                # slice image and mask based on the slices

                image_ = image[:,:,slices_2]
                mask_ = datamask[:,:,slices_2]

                image_ = image_[:,sl,:]
                mask_ = mask_[:,sl,:]

                image_ = image_[slices_0,:,:]
                mask_ = mask_[slices_0,:,:]

                

                # save all images as .npy files         

                file_name = filePath.split(os.sep)[-2] + "_" + str(index) + "_img.npy"
                mask_file_name = filePath.split(os.sep)[-2] + "_" + str(index) + "_mask.npy"

                image_ = image_[0,:,:]
                mask_ = mask_[0,:,:]
                
                image_ = cv2.flip(image_, 1)
                mask_ = cv2.flip(mask_, 1)
                image_ = cv2.rotate(image_, cv2.ROTATE_90_CLOCKWISE)

                # image_ = cv2.resize(image_, (13,13), interpolation = cv2.INTER_AREA)
                # mask_ = cv2.resize(mask_, (13,13), interpolation = cv2.INTER_AREA)
               
                # # Remove all rows in image_ that are all 0
                image_ = image_[~np.all(image_ == 0, axis=1)]
                print(image_.shape)
        
                image_name = filePath.split(os.sep)[-2] + "_" + str(index) + "_img.png"
                mask_name = filePath.split(os.sep)[-2] + "_" + str(index) + "_mask.png"
                cv2.imwrite(path + image_name, image_)
                ##cv2.imwrite(path + mask_name, mask_)
                
                image_ = np.expand_dims(image_, axis=0)
                mask_ = np.expand_dims(mask_, axis=0)

                print(image_)

                np.save(path + file_name, image_)
                np.save(path + mask_file_name, mask_)

                index += 1

    